In [1]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [2]:
from pyvirtualdisplay import Display

virtual_display = Display(size=(800, 600))
virtual_display.start()

In [3]:
# create environment

env_id = "PandaReachDense-v3"
env = gym.make(env_id)

# state & action
s_size = env.observation_space.shape
a_size = env.action_space.shape

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())  # Get a random observation

pybullet build time: Nov 28 2023 23:45:17


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-5.9552684 , -0.35785833, -3.0879943 ], dtype=float32)), ('desired_goal', array([-7.8265357,  2.2923937,  3.950332 ], dtype=float32)), ('observation', array([-1.3140336 , -9.146912  ,  8.239282  , -0.9746187 , -8.799888  ,
        0.43580472], dtype=float32))])


In [4]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  (3,)
Action Space Sample [ 0.39828154 -0.04177831 -0.35688895]


In [5]:
env = make_vec_env(env_id, n_envs=4)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

model = A2C(policy="MultiInputPolicy", env=env, verbose=1)
model.learn(1_000_000)

model.save("a2c_panda_reach")
env.save("vec_normalize.pkl")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 41.7     |
|    ep_rew_mean        | -11.1    |
|    success_rate       | 0.196    |
| time/                 |          |
|    fps                | 468      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 2000   

In [6]:
# Evaluate the agent

from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"
eval_env.training = False
eval_env.norm_reward = False  # not need at test time

model = A2C.load("a2c_panda_reach")
mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/j2y/miniconda3/envs/rl/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -0.27 +/- 0.11
